In [143]:
import os
folder_path = './output_tmp_0/'  # 替换为你的文件夹路径

# 遍历文件夹下的所有文件
def list_files_in_directory(folder_path, suffix):
    file_names = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            if filename.endswith(suffix):
                file_names.append(filename)
    return file_names 

file_names = list_files_in_directory(folder_path, 'json')

In [144]:
import json

# 读取 JSON 文件
data_lst = []
for file_name in file_names:
    with open(folder_path + file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)
        data_lst .append(data)


In [137]:

def load_file_jsonl(file :str):
    data = []
    try:
        with open(file, mode='r', encoding='utf-8') as file:
            for line in file:
                # Parse each line as a JSON object
                json_obj = json.loads(line)
                data.append(json_obj)
    except FileNotFoundError:
        print(f"Error: File not found at {file}")
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format. {e}")
    except Exception as e:
        print(f"Error: {e}")

    return data

In [145]:

def get_input_name(input_output_file_name):
    file = input_output_file_name
    str_lst = file.split('_')
    str_lst = str_lst[:-1]
    tag = str_lst[4]
    model_names = str_lst[4:]
    #index = model_names[-1].rfind('.')
    #model_names[-1] = model_names[-1][:index]
    return '_'.join(model_names)

In [146]:
file_path = folder_path 
files = list_files_in_directory(file_path, 'json')

input_data_lst = []
for file in files:
    input_filename = get_input_name(file)
    input = load_file_jsonl('./new_input/' + input_filename)
    input_data_lst.append(input)

print(input_data_lst)
print(data_lst)

pairs = []
for i in range(len(data_lst)):
    input_output_pair = {}
    input_output_pair['input'] = input_data_lst[i]
    input_output_pair['output'] =  data_lst[i]
    pairs.append(input_output_pair)

[[{'id': 'val-ar-1', 'lang': 'AR', 'model_input': 'كم عدد المقاطعات في جليقية؟', 'model_output_text': 'جليقة، التي تعرف الآن باسم كوريا الجنوبية، تتألف من 16 مقاطعة.', 'model_id': 'arcee-ai/Arcee-Spark', 'soft_labels': [{'start': 27, 'prob': 1.0, 'end': 41}, {'start': 52, 'prob': 1.0, 'end': 54}, {'start': 54, 'prob': 0.3333333333, 'end': 61}], 'hard_labels': [[27, 41], [52, 54]], 'model_output_tokens': ['Ø¬', 'ÙĦÙĬ', 'ÙĤØ©', 'ØĮ', 'ĠØ§ÙĦØªÙĬ', 'ĠØª', 'Ø¹Ø±Ùģ', 'ĠØ§ÙĦØ¢ÙĨ', 'ĠØ¨Ø§Ø³Ùħ', 'ĠÙĥ', 'ÙĪØ±', 'ÙĬØ§', 'ĠØ§ÙĦØ¬ÙĨÙĪØ¨', 'ÙĬØ©', 'ØĮ', 'ĠØª', 'ØªØ£', 'ÙĦÙģ', 'ĠÙħÙĨ', 'Ġ', '1', '6', 'ĠÙħÙĤ', 'Ø§Ø·', 'Ø¹Ø©', '.'], 'model_output_logits': [11.3433694839, 9.0011558533, -0.1627692431, 7.6572833061, 10.1027479172, 8.3447217941, 9.0294656754, 8.9224386215, 4.9082431793, 7.2806453705, 11.7512512207, 7.1793937683, 10.7542667389, 5.6824522018, 8.8103532791, 11.1823453903, 7.4685359001, 1.2530685663, 11.3070726395, 11.582613945, 16.8330955505, 3.8770189285, 1.8877660036, 10.6787233353, 5.34060

In [147]:
def generate_output(input_output_pair):
    input = input_output_pair['input']
    output = input_output_pair['output']
    for i, doc in enumerate(input):
        t = doc['model_output_text']
        doc['soft_labels'] = []
        doc['hard_labels'] = []
        for item in output[i]:
            start = t.find(item)
            end = start + len(item)
            doc['soft_labels'].append({"start": start, "prob": 1.0, "end": end}) 
            doc['hard_labels'].append([start, end])
    return input

new_outputs = []
for pair in pairs:
    new_output = generate_output(pair)
    new_outputs.append(new_output)
print(new_outputs)

[[{'id': 'val-ar-1', 'lang': 'AR', 'model_input': 'كم عدد المقاطعات في جليقية؟', 'model_output_text': 'جليقة، التي تعرف الآن باسم كوريا الجنوبية، تتألف من 16 مقاطعة.', 'model_id': 'arcee-ai/Arcee-Spark', 'soft_labels': [{'start': 0, 'prob': 1.0, 'end': 5}, {'start': 12, 'prob': 1.0, 'end': 16}, {'start': 17, 'prob': 1.0, 'end': 21}, {'start': 22, 'prob': 1.0, 'end': 26}, {'start': 27, 'prob': 1.0, 'end': 32}, {'start': 33, 'prob': 1.0, 'end': 41}, {'start': 52, 'prob': 1.0, 'end': 54}], 'hard_labels': [[0, 5], [12, 16], [17, 21], [22, 26], [27, 32], [33, 41], [52, 54]], 'model_output_tokens': ['Ø¬', 'ÙĦÙĬ', 'ÙĤØ©', 'ØĮ', 'ĠØ§ÙĦØªÙĬ', 'ĠØª', 'Ø¹Ø±Ùģ', 'ĠØ§ÙĦØ¢ÙĨ', 'ĠØ¨Ø§Ø³Ùħ', 'ĠÙĥ', 'ÙĪØ±', 'ÙĬØ§', 'ĠØ§ÙĦØ¬ÙĨÙĪØ¨', 'ÙĬØ©', 'ØĮ', 'ĠØª', 'ØªØ£', 'ÙĦÙģ', 'ĠÙħÙĨ', 'Ġ', '1', '6', 'ĠÙħÙĤ', 'Ø§Ø·', 'Ø¹Ø©', '.'], 'model_output_logits': [11.3433694839, 9.0011558533, -0.1627692431, 7.6572833061, 10.1027479172, 8.3447217941, 9.0294656754, 8.9224386215, 4.9082431793, 7.2806453705, 11.7512512207, 7

In [148]:
import os

# 定义要创建的文件夹路径
new_folder_path = folder_path[:-1] + '_res'  # 替换为你的实际路径

# 创建文件夹
try:
    os.makedirs(new_folder_path)  # 如果文件夹已存在，会引发异常
    print(f"文件夹 '{new_folder_path}' 创建成功！")
except FileExistsError:
    print(f"文件夹 '{new_folder_path}' 已存在。")
except Exception as e:
    print(f"创建文件夹时出错: {e}")

文件夹 './output_tmp_0_res' 创建成功！


In [149]:
mark = folder_path.replace('/', '')
mark = mark.replace('.', '')
for new_output in new_outputs:
    tag = new_output[0]['lang']
    model_name = new_output[0]['model_id']
    with open(new_folder_path + '/' + tag.lower() + '_' + model_name.replace('/', '_') + '.jsonl', 'w', encoding='utf-8') as file:
        for item in new_output:
            json_line = json.dumps(item, ensure_ascii=False)  # 转换为 JSON 字符串
            file.write(json_line + '\n')  # 写入文件并换行